In [1]:
import time
from tqdm import tqdm
import collections
import math
import pickle
from datetime import datetime
import numpy as np
import pandas as pd
import faiss

In [2]:
from loaddata_z import loadData

In [3]:
data_path = "../data/"
_loadData = loadData(data_path)

In [5]:
sample_df = _loadData.get_sample_data()
item_info_df = _loadData.get_item_info()
item_emb_df = _loadData.get_item_emb()
item_emb_df.rename(columns={'click_article_id': 'article_id'}, inplace=True)

In [6]:
item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    
item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)

In [8]:
item_emb_np.shape

(364047, 250)

In [6]:
# 向量检索相似度计算
def embedding_sim(item_emb_df, topk):
    """
    基于内容的文章embedding相似性矩阵计算
    :param click_df: 数据表
    :param item_emb_df: 文章的embedding
    :patam topk: 找最相似的topk篇
    return 文章相似性矩阵
        
    思路: 对于每一篇文章， 基于embedding的相似性返回topk个与其最相似的文章，用了faiss进行加速
    """
    # 文章索引与文章id的字典映射
    item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)
    # 向量进行单位化
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)
    
    item_index = faiss.IndexFlatIP(item_emb_np.shape[1])
    item_index.add(item_emb_np)
    
    # 相似度查询，给每个索引位置上的向量返回topk个item以及相似度
    sim, idx = item_index.search(item_emb_np, topk)
    
    # 将向量检索的结果保存成原始id的对应关系
    item_sim_dict = collections.defaultdict(dict)
    # 将向量检索的结果保存成原始id的对应关系

    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(range(len(item_emb_np)), sim, idx)):
        target_raw_id = item_idx_2_rawid_dict[target_idx]

        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]): 
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value
    
    pickle.dump(item_sim_dict, open('emb_i2i_sim1.pkl', 'wb'))   
    
    return item_sim_dict

In [7]:
item_sim_dict = embedding_sim(item_emb_df, 101)

364047it [01:26, 4185.31it/s]
